In [1]:
import os
import pandas as pd
import re
import requests

In [2]:
df = pd.read_excel('assignment1/response.xlsx')
df['id'] = df['Thư điện tử'].str[:8]
df['folder'] = df.agg('{0[id]}_{0[Họ và đệm]} {0[Tên]}'.format, axis=1)
df = df.sort_values(by=['Được hoàn thành'], ascending=False)

In [3]:
def extract_id(response):
    if type(response) != str:
        return None
    pos = response.find('open?id=')
    if pos == -1:
        pos = response.find('/file/d/')
    if pos != -1:
        pos += 8
        response = response[pos:]
        result = re.search(r"[\w-]+", response)
        return result.group(0)
    else:
        return None

def download_file_from_google_drive(id, destination, replace=False):
    if not replace and os.path.isfile(destination):
        print("Destination file", destination, "exists, download aborted")
        return
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_response(row):
    folder = row['folder']
    response = row['Response 1']    
    data_file_id = extract_id(response)
    if data_file_id is not None:
        destination_dir = os.path.join('assignment1', folder)
        destination = os.path.join(destination_dir, 'data.zip')
        os.makedirs(destination_dir, exist_ok=True)
        print("Downloading", data_file_id, '...')
        download_file_from_google_drive(data_file_id, destination)
        print("Done.")
    else:
        print("Cannot find drive file ID, check response")

def download_by_student_id(student_id):
    for r in df.iterrows():
        r = r[1]
        if r['id'] == student_id:
            print(r['folder'], r['Được hoàn thành'], r['Response 1'])
            download_response(r)
            print('--------------------------------------------------------')
        # break # comment this line to process all assignments

In [4]:
download_by_student_id('17020709')

17020709_Dương Thu Hà 8 Tháng tư 2020  1:54 AM Code: https://github.com/duongghaa/speechProcessing/tree/master/Tuan1_thudulieu

Data: https://drive.google.com/file/d/1lQxeVW4j2PfV4hnML_DskEGFJlk2Jq3k/view?usp=sharing
Destination file assignment1/17020709_Dương Thu Hà/data.zip exists, download aborted
Done.
--------------------------------------------------------
17020709_Dương Thu Hà 6 Tháng tư 2020  12:14 PM Code: https://github.com/duongghaa/speechProcessing/tree/master/Tuan1_thudulieu

Data: https://drive.google.com/file/d/1lQxeVW4j2PfV4hnML_DskEGFJlk2Jq3k/view?usp=sharing
Destination file assignment1/17020709_Dương Thu Hà/data.zip exists, download aborted
Done.
--------------------------------------------------------
17020709_Dương Thu Hà 30 Tháng ba 2020  10:32 PM Code: https://github.com/duongghaa/speechProcessing/tree/master/Tuan1_thudulieu

Data: https://drive.google.com/file/d/1VuMoKIlfNMttHSXH8bOyrWg9_cpFZdwo/view?usp=sharing
Destination file assignment1/17020709_Dương Thu Hà

In [5]:
s = '''
1
3
4
6
7
8
9
10
12
13
14
15
16
17
18
20
22
23
24
26
27
29
30
31
32
33
34
35
36
37
38
39
40
41
42
44
45
46
'''

In [16]:
s='''1
2
3
4
5
6
7
8
9
10
11
14
15
16
17
20
21
22
23
24
25
26
27
30
32
33
35
36
38
39
43
44
45
46
47
48
53
54
56
57
58
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77'''

In [17]:
import random
def pairing(all_id):
    pair = {}
    while len(all_id) > 1:
        id1 = all_id[0]
        id2 = id1
        while id2 == id1:
            id2 = all_id[random.randint(1,len(all_id)-1)]
        print(id1, id2)
        pair[id1] = id2
        pair[id2] = id1
        all_id = list([x for x in all_id if x != id1 and x != id2])
    return pair, all_id
all_id = list([int(x) for x in s.split('\n') if x != ''])
print(all_id)
pair, remain = pairing(all_id)
print("remain", remain)
for x in all_id:
    if x in pair:
        print(f"{x},{pair[x]}")

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 30, 32, 33, 35, 36, 38, 39, 43, 44, 45, 46, 47, 48, 53, 54, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77]
1 54
2 53
3 57
4 70
5 20
6 24
7 75
8 43
9 77
10 21
11 71
14 61
15 72
16 39
17 58
22 25
23 38
26 35
27 48
30 64
32 63
33 56
36 60
44 66
45 65
46 67
47 69
62 74
68 73
remain [76]
1,54
2,53
3,57
4,70
5,20
6,24
7,75
8,43
9,77
10,21
11,71
14,61
15,72
16,39
17,58
20,5
21,10
22,25
23,38
24,6
25,22
26,35
27,48
30,64
32,63
33,56
35,26
36,60
38,23
39,16
43,8
44,66
45,65
46,67
47,69
48,27
53,2
54,1
56,33
57,3
58,17
60,36
61,14
62,74
63,32
64,30
65,45
66,44
67,46
68,73
69,47
70,4
71,11
72,15
73,68
74,62
75,7
77,9
